In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from datetime import datetime as dt

import numpy as np
import pandas as pd

import Utilities.Weather as uw
import Utilities.SnD as us
import APIs.QuickStats as qs

import warnings # supress warnings
warnings.filterwarnings('ignore')

In [41]:
uw.open_w_sel_file()

In [63]:
#region declarations
corn_states=['IA','IL','IN','OH','MO','MN','SD','NE']
years=range(1985,2023)

# Select the Weather Stations
df_w_sel = uw.get_w_sel_df()
df_w_sel = df_w_sel[df_w_sel[uw.WS_COUNTRY_ALPHA] == 'USA']
# df_w_sel = df_w_sel[df_w_sel[uw.WS_STATE_ALPHA] == 'IL']

# Build the Weather DF
w_vars = [uw.WV_PREC,  uw.WV_SDD_30]
in_files = uw.WS_UNIT_ALPHA
out_cols = uw.WS_UNIT_ALPHA
w_df_all = uw.build_w_df_all(df_w_sel, w_vars, in_files, out_cols)

# # Build the Weights
# weights = us.get_USA_prod_weights('CORN', 'STATE', years, corn_states)

# # Weighted DataFrame
# w_w_df_all = uw.weighted_w_df_all(w_df_all, weights, output_column='USA')

# uw.add_Sdd(w_df_all['hist'])

w_df_all['hist']
# w_vars

,IA_Prec,IL_Prec,IN_Prec,OH_Prec,MO_Prec,MN_Prec,SD_Prec,NE_Prec,IA_Sdd30,IL_Sdd30,IN_Sdd30,OH_Sdd30,MO_Sdd30,MN_Sdd30,SD_Sdd30,NE_Sdd30
time,,,,,,,,,,,,,,,,
1985-01-01,6.397065,24.751257,24.211063,7.685587,29.888802,0.367761,0.009242,0.066882,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000
1985-01-02,0.021137,0.364159,1.984503,3.086113,0.468136,0.021630,0.003087,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000
1985-01-03,0.000000,0.123576,0.128540,0.038566,0.038166,0.001129,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000
1985-01-04,0.000000,2.048156,2.942260,0.975676,2.440511,0.000307,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000
1985-01-05,0.005071,0.218555,0.561939,1.303611,0.133436,0.041556,0.009307,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10,2.343426,1.973759,0.016971,0.268474,1.653609,0.416943,1.319011,2.243129,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000
2022-06-11,2.488926,2.269155,2.834657,2.101578,0.799377,2.756599,1.095849,0.041495,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000
2022-06-12,3.646031,1.864520,6.755681,5.125136,1.808156,1.854909,5.226624,1.622734,0.000000,0.000000,0.00000,0.000000,0.882401,0.0,0.000000,2.324451


In [46]:
# DATES: Pollination SDD (Dates are silking 50% -15 and +15 days)
silking_df=qs.get_QS_progress(progress_var='silking',years=years,cols_subset=['week_ending','Value'])

silk_50_pct=us.dates_from_progress(silking_df, sel_percentage=50)
silk_50_pct_CUR_YEAR=pd.Series([dt(uw.CUR_YEAR,d.month,d.day) for d in silk_50_pct['date']]) # Adding current estimate for silking dates
silk_50_pct.loc[uw.CUR_YEAR]= np.mean(silk_50_pct_CUR_YEAR)

start=silk_50_pct['date']+pd.DateOffset(-15)
end=silk_50_pct['date']+pd.DateOffset(15)
pollination_dates=pd.DataFrame({'start':start,'end':end})


# DATES: Regular SDD (Dates are 20 Jun - 15 Sep)
start=[dt(y,6,20) for y in silk_50_pct.index]
end=[dt(y,9,25) for y in silk_50_pct.index]
regular_dates=pd.DataFrame({'start':start,'end':end},index=silk_50_pct.index)

In [13]:
geo,var= 'USA_TempMax'.split('_')
print(geo)
print(var)

USA
TempMax


In [53]:
day=dt(2022,6,10)

sel_ext_df=w_w_df_all[uw.WD_H_GFS].copy()

sel_ext_df['USA_TempSDD']=sel_ext_df['USA_TempMax']
mask=sel_ext_df['USA_TempSDD']>30.0
sel_ext_df['USA_TempSDD'][mask]=sel_ext_df['USA_TempSDD'][mask]-30.0
sel_ext_df['USA_TempSDD'][~mask]=0

w_w_df_ext = uw.extend_with_seasonal_df(w_df, modes=[uw.EXT_MEAN])
w_w_df_ext

M_pollination_sdd = uw.extract_w_windows(w_w_df_ext[['USA_TempSDD']], pollination_dates)
M_pollination_sdd*9/5

M_regular_sdd = uw.extract_w_windows(w_w_df_ext[['USA_TempSDD']], regular_dates)
M_regular_sdd*9/5-M_pollination_sdd*9/5

,USA_TempSDD
1985,10.520536
1986,15.791750
1987,61.975260
1988,179.854116
1989,21.208432
1990,77.205579
1991,61.665397
1992,6.130904
1993,15.367580
1994,19.579622


In [22]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
# SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
SCOPES = ['https://www.googleapis.com/auth/drive']

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
        
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))
except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')

Files:
SD_TempAvg_gfs.csv (15CjUUaTF472P1BQEUNJO_nhjtP_Ac9eg)
SD_TempMin_gfs.csv (14WYBt8JUXgTGpsczpiCTHlumUnSWithy)
SD_TempMax_gfs.csv (10AIb9jeltMrBDk0KOMPeyrpt5q5Zm5Up)
SD_Prec_gfs.csv (1202k-tHWue4xACcgsigklEjkm3ndaQbT)
OH_TempAvg_gfs.csv (13bYHWIhaIWeqMzq57qx4DOO3bp4VSoij)
OH_TempMin_gfs.csv (116q7VJT5GrXy1s3Bil_PTEtOHZEvzBcl)
OH_TempMax_gfs.csv (1-Dr-lK0jHDiQWlKXuwKPJ3I36CaxebL6)
OH_Prec_gfs.csv (11gON3bke-UMJNi-0hBfbeQKtRhTcVadB)
NE_TempAvg_gfs.csv (13dB922LFFDUQdUZGWpgnvPqjnU9SQIPk)
NE_TempMin_gfs.csv (122O1IMp4fiFosbrK_kd2giBMq-9OucwJ)


In [ ]:
import io

In [24]:
import pandas as pd
import Utilities.Weather as uw

In [26]:
test=uw.get_w_sel_df()

In [36]:
cols=list(test.columns)
cols.remove('country_name')
cols

['amuIds',
 'idBlock',
 'country_alpha',
 'country_code',
 'unit_name',
 'unit_alpha',
 'unit_code',
 'state_name',
 'state_alpha',
 'state_code',
 'Prec',
 'TempAvg',
 'TempMin',
 'TempMax',
 'TempSurf',
 'Soil',
 'Humi',
 'VVI']

In [ ]:
file_id = '1-7mnO-RX9GiCWsLqF5ngo0a4Z5o4je3W'
path = 'https://drive.google.com/uc?export=download&id='+file_id
df = pd.read_csv(path)
df

In [ ]:
# https://stackoverflow.com/questions/52135293/google-drive-api-the-user-has-not-granted-the-app-error

In [ ]:
# https://discuss.streamlit.io/t/google-drive-csv-file-link-to-pandas-dataframe/8057

In [ ]:
# https://developers.google.com/drive/api/v2/reference/files/get

In [ ]:
from apiclient import errors
from apiclient import http
# ...

def print_file_metadata(service, file_id):
  """Print a file's metadata.

  Args:
    service: Drive API service instance.
    file_id: ID of the file to print metadata for.
  """

  return(service.files().get(fileId=file_id).execute())


def print_file_content(service, file_id):
  """Print a file's content.

  Args:
    service: Drive API service instance.
    file_id: ID of the file.

  Returns:
    File's content if successful, None otherwise.
  """
  # print (service.files().get_media(fileId=file_id).execute())
  return service.files().get_media(fileId=file_id).execute()


def download_file(service, file_id, local_fd):
  """Download a Drive file's content to the local filesystem.

  Args:
    service: Drive API Service instance.
    file_id: ID of the Drive file that will downloaded.
    local_fd: io.Base or file object, the stream that the Drive file's
        contents will be written to.
  """
  request = service.files().get_media(fileId=file_id)
  media_request = http.MediaIoBaseDownload(local_fd, request)

  while True:
    download_progress, done = media_request.next_chunk()

    if download_progress:
      print ('Download Progress: %d%%' % int(download_progress.progress() * 100))
    if done:
      print ('Download Complete')
      return

In [ ]:
file_id='1dWZkKoknSvOS2PKkxGmIyh5HtYYd1HZ3'

test=pd.read_csv(print_file_content(service,file_id))

# f = open("test.csv", "r", encoding="utf-8")

# download_file(service,file_id, test)
test

In [3]:
import Utilities.Weather as uw

In [17]:
sel_states=['IL','IA']
sel_df=uw.get_w_sel_df()

if True:
    test=sel_df[sel_df['state_alpha'].isin(sel_states)]
else:
    sel_df=sel_df.set_index('state_alpha',drop=False)
    test=sel_df.loc[sel_states]


test


,amuIds,idBlock,country_name,country_alpha,country_code,unit_name,unit_alpha,unit_code,state_name,state_alpha,state_code,Prec,TempAvg,TempMin,TempMax,TempSurf,Soil,Humi,VVI
0,1018108,129,United States,USA,840,Iowa,IA,wip,Iowa,IA,wip,y,NaN,y,y,NaN,NaN,NaN,NaN
1,1018106,129,United States,USA,840,Illinois,IL,wip,Illinois,IL,wip,y,NaN,y,y,NaN,NaN,NaN,NaN
